# MoI ML CNN Experiment

- Train op fashion mnist
- Laat experimenten zien: kies 2 - 3 params, kies een range en voer tests uit. Doel is 93% accuracy.

In [23]:
import torch
import torch.nn as nn
import seaborn as sns
import sys
from pathlib import Path
sys.path.insert(0, "../")

In [24]:
from src.data import make_dataset
from src.models import imagemodels
from src.models import train_model
import gin


In [25]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [26]:
# Load config for this notebook
gin.parse_config_file("cnn.gin")

ParsedConfigFileIncludesAndImports(filename='cnn.gin', imports=[], includes=[])

In [27]:
# Get MNIST data
train_dataloader, test_dataloader = make_dataset.get_MNIST()

In [28]:

# Set some other parameters
import torch.optim as optim
from src.models import metrics
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()
from src.models import train_model

## Voer een experiment uit.


### Configuratie voor het experiment

Sla de gewenste structuur voor de lagen op in een lijst configuraties

In [29]:
from dataclasses import dataclass
@dataclass
class NNLayerConfig:
    """Class for storing config and results for a layer."""
    loss: float
    num_params: int
    filter1_size: int
    filter1_stride: int
    filter1_padding: int
    filter2_size: int
    filter2_stride: int
    filter2_padding: int
    dim_1: int
    dim_2: int

    # TODO calculate input and output dims
    def calc_dim_1():
        return 0
    def calc_dim_2():
        return 0

configs = []  # List of NNLayerConfig objects

# Configuration for the experiment. For now do nothing with stride and padding.
s1 = [1, 2, 3]
s2 = [2, 1, 1]
dim1 = 64
dim2 = 1
for (f1s, f2s) in zip(s1, s2):
    configs.append(NNLayerConfig(   
                                    loss = 0,
                                    num_params = 0,
                                    filter1_size = f1s, 
                                    filter1_stride = 1,
                                    filter1_padding =  0,
                                    filter2_size = f2s,
                                    filter2_stride = 1,
                                    filter2_padding = 0,
                                    dim_1 = dim1,
                                    dim_2 = dim2
                                    ))



Maak een klasse die met een configuratie als input een NN maakt.

In [30]:
32//3


10

In [34]:



# Define model
class ConfigurableNN (nn.Module):

    def __init__(self, config:NNLayerConfig):
        super().__init__()

        self.config = config
        self.num_filters_l1 = 32 # TODO from config
        self.max_pool_ksize = 2 # TODO from config



        # Convolutions are set separately
        self.convolutions = nn.Sequential(
            nn.Conv2d(  1, # First one is always 1
                        self.num_filters_l1, 
                        kernel_size = config.filter1_size,
                        stride = config.filter1_stride,
                        padding = config.filter1_padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=self.max_pool_ksize),
            nn.Conv2d(  self.num_filters_l1,
                        self.num_filters_l1 * 2,
                        kernel_size = config.filter2_size,
                        stride = config.filter2_stride,
                        padding = config.filter2_padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=self.max_pool_ksize)

            # Werkend voorbeeld:
            # nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),
            # nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),
            # nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),


        )
        
        # Dense is default for now.
        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 64),
            # nn.ReLU(),
            # nn.Linear(64, 32),
            # nn.ReLU(),
            # nn.Linear(32, 10)
        )

    # Forward is default for now.
    def forward(self, x):
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


Loop door de configuraties heen

In [35]:
num_epochs = 2 # During development keep very low. 10 is for real tests
learning_rate = 1e-3

# For to be debugging:
x, y = next(iter(train_dataloader))
x.shape, y.shape

print("input shape is:")
print(x.shape)

for config in configs:
    model = ConfigurableNN(config).to(device)
    print("================")
    print("STARTING NEW MODEL")
    print("================")
    print(model)

    out = x
    for conv in model.convolutions:
        print("-----------------")
        print("Convolution:")
        print(conv)
        out = conv(out)
        print(out.shape)
    print("-----------------")
    print("Examining dense:")
    print("-----------------")
    for dense in model.dense:
        print("-----------------")
        print("Dense:")
        print(dense)
        out = dense(out)
        print(out.shape)
    break
    model = train_model.trainloop(
        epochs=num_epochs,
        model=model,
        optimizer=optimizer,
        learning_rate=learning_rate,
        loss_fn=loss_fn,
        metrics=[accuracy],
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        log_dir="../models/test/",
        train_steps=len(train_dataloader),
        eval_steps=len(test_dataloader),
    )
    break # During development
    

input shape is:
torch.Size([32, 1, 28, 28])
STARTING NEW MODEL
ConfigurableNN(
  (convolutions): Sequential(
    (0): Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=32, out_features=64, bias=True)
  )
)
-----------------
Convolution:
Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
torch.Size([32, 32, 28, 28])
-----------------
Convolution:
ReLU()
torch.Size([32, 32, 28, 28])
-----------------
Convolution:
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
torch.Size([32, 32, 14, 14])
-----------------
Convolution:
Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
torch.Size([32, 64, 13, 13])
-----------------


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x2304 and 32x64)

In [ ]:

# Loop through some kernel sizes
for ksize in range(2,4):

    
    model = ExpCNN().to(device)
    model.set_convolution(nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=ksize, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),


        ))

    # Now train
    model = train_model.trainloop(
        epochs=2, # was 10
        model=model,
        optimizer=optimizer,
        learning_rate=1e-3,
        loss_fn=loss_fn,
        metrics=[accuracy],
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        log_dir="../models/test/",
        train_steps=len(train_dataloader),
        eval_steps=len(test_dataloader),
    )


Checkout results using tensorboard: 
- run `tensorboard --logdir models` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`